# Using Neptune with Tensorflow

In this example, we will use Neptune to log meta-data generated from training using Tensorflow.

By the end of this guide, you will be able to
* Track and version the data.
* Log losses and other metrics generated from training.
* Log prediction over multiple epochs.
* Save the generated model with model registry.

[See this example in Neptune](https://app.neptune.ai/o/common/org/data-versioning/experiments?compare=IwdgNMQ&split=tbl&dash=artifacts&viewId=6777136b-938e-4639-943d-3f6bc52f8497)

## Before you start

This notebook example lets you try out Neptune anonymously, with zero setup.

* If you're running the notebook on your local machine, you need to have [Python](https://www.python.org/downloads/) and [pip](https://pypi.org/project/pip/) installed.
* If you want to see the example logged to your own workspace instead:
    * Create a Neptune account → [Take me to registration](https://neptune.ai/register)
    * Create a Neptune project that you will use for tracking metadata → [Tell me more about projects](https://docs.neptune.ai/administration/projects)

## Install Neptune and dependencies

In [ ]:
! pip install neptune-client tensorflow==2.9.2 numpy==1.21.6

## Start a run

To connect your script to Neptune and create a new run, we tell Neptune:
* **Who you are** - with a Neptune API token
* **Where to send your data** - to a Neptune project

The cell below lets you record data to the public project [common/quickstarts](https://app.neptune.ai/common/quickstarts) as an anonymous user.

In [ ]:
import neptune.new as neptune

run = neptune.init_run(
    api_token=neptune.ANONYMOUS_API_TOKEN,
    project="common/quickstarts",
)

Alternatively, you can log the example to your own workspace.

To do that, replace the code above with the following:

```python
from getpass import getpass

run = neptune.init_run(
    api_token=getpass("Enter your Neptune API token: "),
    project="workspace-name/project-name",  # replace with your own
)
```

For example, if your workspace name is `ml-team` and the project name is `classification`, the project argument is: `project="ml-team/classification"`.

To find your API token and project name, [log in to Neptune](https://app.neptune.ai/).
- In the top-right corner, click your avatar and select **Get your API token**.
- To find and copy your project name, navigate to the project, then click **Settings** → **Properties**.

---

You now have new run in Neptune! From here on, we'll use the `run` object to log metadata.

**To open the run in Neptune, follow the link that appeared in the cell output.**

There's not much to display yet, but keep the tab with the run open to see what happens next.

## Log metadata to Neptune



Import libraries

In [ ]:
import io

import tensorflow as tf
import numpy as np

Download the MNIST Data

In [ ]:
!wget 'https://storage.googleapis.com/tensorflow/tf-keras-datasets/mnist.npz'

(Neptune) Track and version data files used for training

In [ ]:
run['data/details'].track_files('mnist.npz')

In [ ]:
with np.load('mnist.npz') as data:
  train_examples = data['x_train']
  train_labels = data['y_train']
  test_examples = data['x_test']
  test_labels = data['y_test']

Normalize Data for training

In [ ]:
def normalize_img(image):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255.

train_examples = normalize_img(train_examples)
test_examples = normalize_img(test_examples)

Prepare data for training

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_examples, train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_examples, test_labels))

train_dataset = train_dataset.shuffle(params['shuffle_buffer_size']).batch(params['batch_size'])
test_dataset = test_dataset.batch(params['batch_size'])

Parameters for training

In [ ]:
params = {'batch_size': 1024,
          'shuffle_buffer_size': 100,
          'lr': 0.001,
          'num_epochs': 10,
          'num_visualization_examples': 10}

(Neptune) Log training parameters

In [ ]:
run['training/model/params'] = params

Prepare model

In [ ]:
# Model
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10)
])

# Loss
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Optimizer
optimizer=tf.keras.optimizers.Adam(params['lr'])

(Neptune) Log model summary

In [ ]:
with io.StringIO() as s: 
  model.summary(print_fn=lambda x: s.write(x + '\n'))
  model_summary = s.getvalue()

run['training/model/summary'] = model_summary

Helper functions for training loop

In [ ]:
def loss_and_preds(model, x, y, training):
  # training=training is needed only if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  y_ = model(x, training=training)

  return loss_object(y_true=y, y_pred=y_), y_

def grad(model, inputs, targets):
  with tf.GradientTape() as tape:
    loss_value, _ = loss_and_preds(model, inputs, targets, training=True)
  return loss_value, tape.gradient(loss_value, model.trainable_variables)

Training Loop

In [ ]:
for epoch in range(params['num_epochs']):
  epoch_loss_avg = tf.keras.metrics.Mean()
  epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

  for x, y in train_dataset:
    loss_value, grads = grad(model, x, y)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    epoch_loss_avg.update_state(loss_value)
    epoch_accuracy.update_state(y, model(x, training=True))

  # (neptune) Log metrics for the epoch
  # Train metrics
  run['training/train/loss'].log(epoch_loss_avg.result())
  run['training/train/accuracy'].log(epoch_accuracy.result())

  # (neptune) Log test metrics
  test_loss, test_preds = loss_and_preds(model, test_examples, test_labels, False)
  run['training/test/loss'].log(test_loss)
  acc = epoch_accuracy(test_labels, test_preds)
  run['training/test/accuracy'].log(acc)

  # (neptune) Log test prediction
  for idx in range(params['num_visualization_examples']):
    np_image = test_examples[idx].numpy().reshape(28, 28)
    image = neptune.types.File.as_image(np_image)
    pred_label = test_preds[idx].numpy().argmax()
    true_label = test_labels[idx]
    run[f'training/visualization/epoch_{epoch}'].log(image, description=f'pred={pred_label} | actual={true_label}')


  if epoch % 5 == 0 or epoch == (params['num_epochs'] - 1):
    print("Epoch {:03d}: Loss: {:.3f}, Accuracy: {:.3%}".format(epoch,
                                                                epoch_loss_avg.result(),
                                                                epoch_accuracy.result()))

## Stop logging

Once you are done logging, stop tracking the run.

In [ ]:
run.stop()

## Explore the results in Neptune

<font color=red>[Add steps to view the results in the app.]</font>

You can also check out an [example run](https://app.neptune.ai/o/common/org/showroom/e/SHOW-37/charts). <font color=red>[Edit link]</font>

# Summary <font color=red>(optional)</font>

<font color=red>[If the example is long, a summary of what was learned.]</font>



# Next steps
<font color=red>[Tips and next steps. Related links.]</font>